In [ ]:
# @title Chronolocate a Shadow 📅 { display-mode: "form" }

# @markdown ### ⬅️ Click to find possible times that match the below information

# @markdown Estimate the angles to the sun (in degrees):
min_elevation_angle = 60  # @param {type:"number"} In degrees
max_elevation_angle = 70  # @param {type:"number"} In degrees

min_azimuth_angle = 170  # @param {type:"number"} In degrees
max_azimuth_angle = 180  # @param {type:"number"} In degrees


latlon = "40.7600000,-73.9840000" # @param {type:"string"}
year=2025  # @param {type:"number"}
utc_offset = -5  # @param {type:"number"}
dst=0  # @param {type:"number"}
step_minutes = 15  # @param {type:"number"}


import pandas as pd

## Get the shadow data from sunearthtools
data_url = f"https://www.sunearthtools.com/tools/downloadFile.php?tableformat=3&point={latlon}&utc={utc_offset}&yearAT={year}&dstAT={dst}&step={step_minutes}&email="
df = pd.read_csv(data_url, sep=';', na_values=['--'])


## Create a new dataframe where each row corresponds to a time step

# rename first column to date
df.rename(columns={df.columns[0]: 'date'}, inplace=True)

cols = df.columns[1:-1] # first column is date, last columns is empty

# Create a new DataFrame with the desired structure
new_df = pd.DataFrame(columns=['date','time', 'elevation', 'azimuth'])

for i in range(0, len(cols), 2):
    time = cols[i].split(' ')[1]  # Extract time from column name
    elevation = df[cols[i]].values
    azimuth = df[cols[i + 1]].values

    # Create a temporary DataFrame for this time step
    temp_df = pd.DataFrame({
        'date': df['date'].values,
        'time': [time] * len(df),
        'elevation': elevation,
        'azimuth': azimuth
    })

    # Append to the new DataFrame
    dfs = [new_df, temp_df]
    # Keep only DataFrames that are not empty and not all-NA
    dfs = [df for df in dfs if not df.empty and not df.isna().all().all()]
    if dfs:
        new_df = pd.concat(dfs, ignore_index=True)
    else:
        new_df = pd.DataFrame()

new_df.dropna(inplace=True)
new_df.sort_values(by=['date', 'time'], inplace=True)


## Filter the DataFrame based on azimuth and elevation
filtered_df = new_df[
    (new_df['azimuth'] >= min_azimuth_angle) &
    (new_df['azimuth'] <= max_azimuth_angle) &
    (new_df['elevation'] >= min_elevation_angle) &
    (new_df['elevation'] <= max_elevation_angle)
    ].copy()

# Reset index for the filtered DataFrame
filtered_df.reset_index(drop=True, inplace=True)
filtered_df['date'] = pd.to_datetime(filtered_df['date'], format='%Y-%m-%d')

# For each date find the min and the max time
result_df = filtered_df.groupby('date').agg(
    min_time=('time', 'min'),
    max_time=('time', 'max')
).reset_index()

# For simple summary group by contiguous sets of dates
result_df['day_delta'] = result_df['date'].diff().dt.days.fillna(1).astype(int)
result_df['contiguous'] = (result_df['day_delta'] > 1).cumsum()

result_summary = result_df.groupby(['contiguous']).agg(
    min_date=('date', 'min'),
    max_date=('date', 'max'),
    min_time=('min_time', 'min'),
    max_time=('max_time', 'max')
).reset_index(drop=True)

print(result_summary.to_string(index=False))